In [1]:
import torch
from torch.utils.data import DataLoader

import numpy as np

from rpn.build_rpn import RPN_Model
from sam.build_sam import SAM_Model
from transformers import pipeline

from data_builder.build_dataset import PlanetscopeDataset

import os
import time
import json

from eval_utils import filter_boxes, calculate_iou, calculate_precision_recall, filter_masks

import pandas as pd
from rpn import _transforms as T
from torch.utils.data import ConcatDataset

/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_transform(image_enhancement="FALSE"):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    if image_enhancement == "TRUE":
        transforms.append(T.ContrastBasedAdaptiveGammaCorrection())
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

#### Read and Print the evaluation config from the JSON file

In [3]:
#Create a folder with unix timestamp
root_path = os.getcwd() + '/results/' + str(int(time.time())) + '/'
os.mkdir(root_path)

with open('eval_config.json', 'r') as f:
    eval_config = json.load(f)


print("Evaluation Configuration")
print(json.dumps(eval_config, indent=1))

Evaluation Configuration
{
 "DATASET1_PATH": "/scratch/aghosh57/SAT-SAM(Dataset)/ps_france/all_dataset/",
 "DATASET1": "FRANCE",
 "DATASET2_PATH": "/scratch/aghosh57/SAT-SAM(Dataset)/ps_france/all_dataset/",
 "DATASET2": "RWANDA",
 "MODEL": "MASKRCNN",
 "IMAGE_ENHANCEMENT": "FALSE",
 "FILTRATION": "TRUE",
 "TRAIN_TYPE": "FINETUNE",
 "NMS_THRESHOLD": 0.9,
 "PRED_CONFIDENCE_THRESHOLD": 0.6,
 "ENSEMBLE_BOX_OVERLAP_THRESHOLD": 0.5,
 "ENSEMBLE_BOX_BETA": 0.4,
 "RPN_MODEL_PATH": "rpn/checkpoints/1692043128/rpn_model_1.09.pth",
 "SAM_MODEL_PATH": "sam/checkpoint/sam_vit_l_0b3195.pth"
}


#### Set the device and load the dataset for evaluation

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset1_test = PlanetscopeDataset(eval_config['DATASET1_PATH'], get_transform(image_enhancement=eval_config['IMAGE_ENHANCEMENT']))
dataset2_test = PlanetscopeDataset(eval_config['DATASET2_PATH'], get_transform(image_enhancement=eval_config['IMAGE_ENHANCEMENT']))

dataset_test = ConcatDataset([dataset1_test, dataset2_test])

torch.manual_seed(1)
indices = torch.randperm(len(dataset_test)).tolist()
dataset_test = torch.utils.data.Subset(dataset_test, indices[-int(len(indices)*0.2):])

#### Load the proper model to be used for evaluation

In [5]:
if eval_config['MODEL'] == 'SAT-SAM':
    rpn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device, eval_config['TRAIN_TYPE'])
    sam_model = SAM_Model(eval_config['SAM_MODEL_PATH'], 'large', device)
elif eval_config['MODEL'] == 'SAM':
    vanilla_sam_model = pipeline("mask-generation", model="facebook/sam-vit-large", device=device)
elif eval_config['MODEL'] == 'MASKRCNN':
    maskrcnn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device, eval_config['TRAIN_TYPE']) #Load the Pre-Trained MaskRCNN model

In [6]:
results = pd.DataFrame(columns=['parcel_id', 'parcel_path', 'gt_mask_ct', 'pred_mask_ct', 'mean_iou', 'p_50', 'r_50', 'p_70', 'r_70', 'p_90', 'r_90'])

for i, (id, sam_image, rpn_image, target, ensemble, path)  in enumerate(dataset_test): 
    try:
        if eval_config['MODEL'] == 'SAT-SAM':
            print('SAT-SAM')
            rpn_image = rpn_image.squeeze(0).to(device)  
            predictions = rpn_model.predict(rpn_image)
            predictions = rpn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
            
            if eval_config['FILTRATION'] == 'TRUE':
                filtered_predictions = filter_boxes(predictions, ensemble, eval_config['ENSEMBLE_BOX_BETA'], eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
                low_res_masks, iou_predictions = sam_model.predict(sam_image, filtered_predictions)
            else:
                low_res_masks, iou_predictions = sam_model.predict(sam_image, predictions['boxes'])
            
            high_res_masks = sam_model.postprocess(low_res_masks, tuple(sam_image.size))
            pred_masks = high_res_masks.squeeze().cpu().numpy()
        
        elif eval_config['MODEL'] == 'SAM':
            print('SAM')
            outputs = vanilla_sam_model(sam_image, points_per_batch=32)
            if eval_config['FILTRATION'] == 'TRUE':
                pred_masks = filter_masks(outputs['masks'], ensemble, eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
            else:
                pred_masks = outputs["masks"]
        
        elif eval_config['MODEL'] == 'MASKRCNN':
            print('MASKRCNN')
            rpn_image = rpn_image.squeeze(0).to(device)
            predictions = maskrcnn_model.predict(rpn_image)
            predictions = maskrcnn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
            if eval_config['FILTRATION'] == 'TRUE':
                pred_masks = filter_masks(np.squeeze(np.array(predictions['masks'])), ensemble, eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
            else:
                pred_masks = predictions['masks']

        iou_score, iou_matrix = calculate_iou(target_masks=np.array(target['masks']), predicted_masks=np.array(pred_masks))
        
        p_50, r_50 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.5)
        p_70, r_70 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.7)
        p_90, r_90 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.9)
        
        # Round off the values to 2 decimal places
        iou_score = round(iou_score, 2)
        p_50, r_50 = round(p_50, 2), round(r_50, 2)
        p_70, r_70 = round(p_70, 2), round(r_70, 2)
        p_90, r_90 = round(p_90, 2), round(r_90, 2)

        print("Image Id: ", id, " Average IoU Score: ", iou_score)
        print("IoU Matrix: ", iou_matrix)
        print("Precision: ", p_50, p_70, p_90)
        print("Recall: ", r_50, r_70, r_90)

        results.loc[i] = [id, path, len(target['masks']), len(pred_masks), iou_score, p_50, r_50, p_70, r_70, p_90, r_90]
    except:
        print("Error in image: ", id)
        continue

MASKRCNN
num_target_masks:  14
num_predicted_masks:  11
row_ind:  [ 0  1  2  3  4  5  6  7  9 10 13]
col_ind:  [ 8  3  2  5  4  6  0 10  7  1  9]
Image Id:  2072  Average IoU Score:  0.34
IoU Matrix:  [0.63555514 0.3541067  0.20710749 0.40385538 0.60193128 0.
 0.20470085 0.09928125 0.71977593 0.52880307 0.03384468]
Precision:  0.36 0.09 0.0
Recall:  0.29 0.07 0.0
MASKRCNN
num_target_masks:  13
num_predicted_masks:  12
row_ind:  [ 0  1  2  3  4  5  6  7  8  9 11 12]
col_ind:  [ 2 11  7  9  8  6  1  3  5  0  4 10]
Image Id:  1306  Average IoU Score:  0.44
IoU Matrix:  [0.47090345 0.         0.58486842 0.47664511 0.43481013 0.
 0.47415634 0.49706745 0.52719684 0.53281352 0.42449579 0.86168767]
Precision:  0.33 0.08 0.0
Recall:  0.31 0.08 0.0
MASKRCNN
num_target_masks:  16
num_predicted_masks:  12
row_ind:  [ 0  1  2  3  4  5  9 11 12 13 14 15]
col_ind:  [ 7  4  6 10  1  3  2  0  9  5 11  8]
Image Id:  3363  Average IoU Score:  0.23
IoU Matrix:  [0.62941355 0.         0.09101678 0.03483724

In [7]:
mean_IoU = results['mean_iou'].mean()
eval_config['MEAN_IOU'] = mean_IoU

In [8]:
results.to_csv(root_path + 'results', index=False)

# save model and configuration
with open(root_path + 'eval_config.json', 'w') as f:
    json.dump(eval_config, f, indent=1)
    f.close()
print("Evaluation Configuration saved to " + root_path + 'train_config.json') 

Evaluation Configuration saved to /home/aghosh57/Kerner-Lab/SAT-SAM/results/1692329259/train_config.json
